In [1]:
import pandas as pd
import numpy as np

In [2]:
events = pd.read_csv('user_activity.csv')
structure = pd.read_csv('structure.csv')
ones = pd.read_csv('ones_only.csv')
targets = pd.read_csv('targets.csv')
events_test = pd.read_csv('user_activity_test.csv')

# Task 3

In [3]:
Y = targets.passed.values
user_ids_train = targets.user_id.values
user_all_score_train = targets.score.values

In [4]:
%%time

last_activity = np.max(events.time.values)

user_achived_score_train = np.array([])
user_discovered_train = np.array([])
user_passed_train = np.array([])
user_not_doing_anything_train = np.array([])

for user in targets.user_id.values:
    user_achived_score_train = np.append(user_achived_score_train, np.sum(events[events.user_id==user][events.action=='passed'].step_cost.values))
    user_passed_train = np.append(user_passed_train, events[events.user_id==user][events.action=='passed'].shape[0])
    user_discovered_train = np.append(user_discovered_train, events[events.user_id==user][events.action=='discovered'].shape[0])
    
    last_user_activity = np.max(events[events.user_id==user]['time'].values)
    user_not_doing_anything_train = np.append(user_not_doing_anything_train, last_activity - last_user_activity)
    
    if user % 1000 < 3:
        print('user ' + str(user) + ' finished')

/home/alexander/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/alexander/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/alexander/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


user 1 finished
user 2 finished
user 1001 finished
user 2000 finished
user 2001 finished
user 2002 finished
user 3001 finished
user 3002 finished
user 4000 finished
user 4001 finished
user 4002 finished
user 5001 finished
user 5002 finished
user 6000 finished
user 6001 finished
user 6002 finished
user 7000 finished
user 7001 finished
user 7002 finished
user 8000 finished
user 8001 finished
user 8002 finished
user 9000 finished
user 9002 finished
user 10001 finished
user 10002 finished
user 11000 finished
user 11001 finished
user 11002 finished
user 12000 finished
user 12001 finished
user 12002 finished
user 14000 finished
user 14001 finished
user 14002 finished
user 15000 finished
user 15001 finished
user 15002 finished
user 16000 finished
user 16001 finished
user 17000 finished
user 18000 finished
user 19000 finished
user 19001 finished
user 19002 finished
user 20000 finished
user 20001 finished
user 20002 finished
CPU times: user 1h 33min 41s, sys: 2.7 s, total: 1h 33min 44s
Wall tim

In [6]:
X = pd.DataFrame()
# X['all_score'] = user_all_score_train
X['score_by_events'] = user_achived_score_train
X['passed_count'] = user_passed_train
X['discovered_count'] = user_discovered_train
X['did_nothing'] = user_not_doing_anything_train

In [7]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [51]:
Xtrain, Xval, Ytrain, Yval = train_test_split(X, Y, test_size=0.1, random_state=42)

In [59]:
model = GradientBoostingClassifier(max_depth=20, learning_rate=1.25)

In [60]:
model.fit(X, Y)

GradientBoostingClassifier(init=None, learning_rate=1.25, loss='deviance',
              max_depth=20, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [61]:
print(f1_score(Yval, model.predict(Xval)))

1.0


In [62]:
%%time

last_activity = np.max(events_test.time.values)

user_achived_score_test = np.array([])
user_discovered_test = np.array([])
user_passed_test = np.array([])
user_not_doing_anything_test = np.array([])

for user in events_test.groupby('user_id').sum().index:
    user_achived_score_test = np.append(user_achived_score_test, np.sum(events_test[events_test.user_id==user][events_test.action=='passed'].step_cost.values))
    user_passed_test = np.append(user_passed_test, events_test[events_test.user_id==user][events_test.action=='passed'].shape[0])
    user_discovered_test = np.append(user_discovered_test, events_test[events_test.user_id==user][events_test.action=='discovered'].shape[0])
    
    last_user_activity = np.max(events_test[events_test.user_id==user]['time'])
    user_not_doing_anything_test = np.append(user_not_doing_anything_test, last_activity - last_user_activity)
    
    if user % 1000 < 3:
        print('user ' + str(user) + ' finished')

/home/alexander/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/alexander/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/alexander/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


user 1000 finished
user 1002 finished
user 3000 finished
user 5000 finished
user 9001 finished
user 10000 finished
user 13000 finished
user 13001 finished
user 13002 finished
user 16002 finished
user 17001 finished
user 17002 finished
user 18001 finished
user 18002 finished
CPU times: user 5min 44s, sys: 207 ms, total: 5min 44s
Wall time: 5min 45s


In [63]:
Xtest = pd.DataFrame()
Xtest['score_by_events'] = user_achived_score_test
Xtest['passed_count'] = user_passed_test
Xtest['discovered_count'] = user_discovered_test
Xtest['did_nothing'] = user_not_doing_anything_test

In [64]:
def create_submission(X, name):
    np.savetxt('%s.csv'%name, X, delimiter=',', fmt="%d", header='user_id,passed', comments='')

In [65]:
ans = model.predict(Xtest)
result = np.concatenate((np.asarray(events_test.groupby('user_id').sum().index, dtype=int).reshape(-1, 1), np.asarray(ans, dtype=int).reshape(-1, 1)), axis=1)
create_submission(result, 'submission')